# 02: Missing Values

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Load the data
data = pd.read_csv('../datasets/melb_data.csv')

# Set the target column
y = data.Price

# use only numerical features for now 
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# divide data into train/valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Method 1: Drop columns with missing values

In [6]:
cols_with_missing = [col for col in X_train.columns 
                     if X_train[col].isnull().any()]

reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print('MAE after dropping columns')
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE after dropping columns
183550.22137772635


#### Method 2: Imputation

In [14]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(strategy='mean')
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train), columns=X_train.columns)
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid), columns=X_valid.columns)

print('MAE with imputation')
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE with imputation
178166.46269899711


#### Method 3: Extenstion to imputation

In [ ]:
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# new column to indicate if imputed
for col in cols_with_missing:
    